In [1]:
import os
import sys
os.environ['SPARK_HOME']='/home/samar/spark'
os.environ['PYLIB']=os.environ['SPARK_HOME']+'/python/lib'
sys.path.insert(0,os.environ['PYLIB']+'/py4j-0.10.9-src.zip')
sys.path.insert(1,os.environ['PYLIB']+'/pyspark.zip')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("AssignmentAnswers").getOrCreate()

In [4]:
sc = spark.sparkContext
print(f'spark version {spark.version} and sc version {sc.version}')

spark version 3.0.1 and sc version 3.0.1


Q1. Find all words in shakespare.txt that have a length greater than ten. Find out the count for each of these words by creating a rdd which will have elements such as ('somelongword',23) etc. where 23 is the number of somelongword present in the data. Find out the sum of the product of word length and word count for these pairs.
Provide the code and the final answer

In [5]:
fileloc = 'file:///home/samar/shakespeare.txt'

In [42]:
sc.textFile('file:///home/samar/shakespeare.txt').flatMap(lambda x: x.split(' ')).filter(lambda x: len(x) > 10)\
.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)\
.map(lambda x: (len(x[0]) * x[1])).reduce(lambda x, y: x + y)

173338

Q2. Save the first ten thousand lines of apachelogs in a file apachelogs10k. Copy this file in HDFS. Find out the average request size per host for the days present in this file. You will have to compute the number of hosts by day and the total request size by day and divide the total request size by number of hosts to come to this

In [7]:
apachelogs10k = sc.textFile('apachelogs10k')
apachelogs10k.first()

'in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839'

In [8]:
import re
import datetime

from pyspark.sql import Row

month_map = {'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
    'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12}

def parse_apache_time(s):
    """ Convert Apache time format into a Python datetime object
    Args:
        s (str): date and time in Apache time format
    Returns:
        datetime: datetime object (ignore timezone for now)
    """
    return datetime.datetime(int(s[7:11]),
                             month_map[s[3:6]],
                             int(s[0:2]),
                             int(s[12:14]),
                             int(s[15:17]),
                             int(s[18:20]))


def parseApacheLogLine(logline):
    """ Parse a line in the Apache Common Log format
    Args:
        logline (str): a line of text in the Apache Common Log format
    Returns:
        tuple: either a dictionary containing the parts of the Apache Access Log and 1,
               or the original invalid log line and 0
    """
    match = re.search(APACHE_ACCESS_LOG_PATTERN, logline)
    if match is None:
        return (logline, 0)
    size_field = match.group(9)
    if size_field == '-':
        size = int(0)
    else:
        size = int(match.group(9))
    return (Row(
        host          = match.group(1),
        client_identd = match.group(2),
        user_id       = match.group(3),
        date_time     = parse_apache_time(match.group(4)),
        method        = match.group(5),
        endpoint      = match.group(6),
        protocol      = match.group(7),
        response_code = int(match.group(8)),
        content_size  = size
    ), 1)

In [9]:
# A regular expression pattern to extract fields from the log line
APACHE_ACCESS_LOG_PATTERN = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)" (\d{3}) (\S+)'

In [10]:
import sys
import os

logFile = "hdfs://localhost:8020/user/samar/apachelogs10k"
# logFileRDD = sc.textFile(fileLoc)
# logFile = os.path.join(baseDir, inputPath)

def parseLogs():
    """ Read and parse log file """
    parsed_logs = (sc
                   .textFile(logFile)
                   .map(parseApacheLogLine)
                   .cache())

    access_logs = (parsed_logs
                   .filter(lambda s: s[1] == 1)
                   .map(lambda s: s[0])
                   .cache())

    failed_logs = (parsed_logs
                   .filter(lambda s: s[1] == 0)
                   .map(lambda s: s[0]))
    failed_logs_count = failed_logs.count()
    if failed_logs_count > 0:
        print('Number of invalid logline: %d' % failed_logs.count())
        for line in failed_logs.take(20):
            print('Invalid logline: %s' % line)

    print('Read %d lines, successfully parsed %d lines, failed to parse %d lines' % 
          (parsed_logs.count(), access_logs.count(), failed_logs.count()))
    return parsed_logs, access_logs, failed_logs


parsed_logs, access_logs, failed_logs = parseLogs()

Read 10000 lines, successfully parsed 10000 lines, failed to parse 0 lines


In [11]:
dailyHosts = access_logs.map(lambda x: (x.date_time.day,x.host)).distinct().\
map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y)
dailyHosts.take(2)

[(1, 853)]

In [12]:
dailySize = access_logs.map(lambda x: (x.date_time.day, x.content_size)).reduceByKey(lambda x, y: x + y)
dailySize.take(10)

[(1, 179517299)]

In [13]:
AvgDailyRequestSizePerHost = dailyHosts.join(dailySize).map(lambda x: (x[0], x[1][1]/x[1][0]))

In [14]:
AvgDailyRequestSizePerHost.collect()

[(1, 210454.04337631888)]

3-6 - for these exercises use the following list as the data 
['INFY,2018-01-01,100,100071', 'INFY,2018-01-02,120,107130', 'INFY,2018-01-03,110,124175', 'INFY,2018-01-02,115,109175', 'BHEL,2018-01-01,4005,21000', 'BHEL,2018-01-02,5240,22015', 'BHEL,2018-01-03,5287,23450', 'BHEL,2018-01-04,6480,28156']

Q3. Create a RDD from the above list and map it to tuple of 4 by splitting each element on comma

In [15]:
StocksList = ['INFY,2018-01-01,100,100071', 'INFY,2018-01-02,120,107130', 
              'INFY,2018-01-03,110,124175', 'INFY,2018-01-02,115,109175', 
              'BHEL,2018-01-01,4005,21000', 'BHEL,2018-01-02,5240,22015', 
              'BHEL,2018-01-03,5287,23450', 'BHEL,2018-01-04,6480,28156']
StocksRDD = sc.parallelize(StocksList).map(lambda x: x.split(',')).map(
lambda x: (x[0],x[1],int(x[2]),float(x[3])))
StocksRDD.take(2)

[('INFY', '2018-01-01', 100, 100071.0), ('INFY', '2018-01-02', 120, 107130.0)]

Q4. Define a schema to represent the dataframe that you will create from the rdd. Give the schema fields - stock of type string, date of type string, qty of type int, vlu of type double. Use the rdd and schema to create a dataframe.

In [16]:
from pyspark.sql.types import *

In [17]:
stocks_schema = StructType([StructField('stock', StringType()),
                           StructField('date', StringType()),
                           StructField('qty', IntegerType()),
                           StructField('vlu', DoubleType())])

In [18]:
stocks_df = spark.createDataFrame(StocksRDD, stocks_schema)

Q5. Create a temporary table|view for this dataframe. And find the percentile for quantity traded (qty) by stock for this table, dataframe.

In [19]:
stocks_df.createOrReplaceTempView('stocktbl')

In [40]:
spark.sql('''
    select stock,date,qty, percent_rank() over(partition by stock order by qty) as stock_qty_percentile,
    vlu
    from stocktbl 
''').show()

+-----+----------+----+--------------------+--------+
|stock|      date| qty|stock_qty_percentile|     vlu|
+-----+----------+----+--------------------+--------+
| INFY|2018-01-01| 100|                 0.0|100071.0|
| INFY|2018-01-03| 110|  0.3333333333333333|124175.0|
| INFY|2018-01-02| 115|  0.6666666666666666|109175.0|
| INFY|2018-01-02| 120|                 1.0|107130.0|
| BHEL|2018-01-01|4005|                 0.0| 21000.0|
| BHEL|2018-01-02|5240|  0.3333333333333333| 22015.0|
| BHEL|2018-01-03|5287|  0.6666666666666666| 23450.0|
| BHEL|2018-01-04|6480|                 1.0| 28156.0|
+-----+----------+----+--------------------+--------+



Q6. Use arrow pandas_udf to find sum and average of vlu for the above table.

In [27]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.sql.execution.arrow.pyspark.fallback.enabled", "true")

In [36]:
from pyspark.sql.functions import *
import pandas as pd

@pandas_udf("double")
def sum_udf(v: pd.Series) -> float:
    return v.sum()

@pandas_udf('double')
def avg_udf(v: pd.Series) ->  float:
    return v.mean()

In [37]:
stocks_df.select(sum_udf('vlu').alias('totalvlu'), avg_udf('vlu').alias('avgvlu')).show()

+--------+-------+
|totalvlu| avgvlu|
+--------+-------+
|535172.0|66896.5|
+--------+-------+



In [38]:
stocks_df.groupBy('stock').\
agg(sum_udf('vlu').alias('totalvlu'), avg_udf('vlu').alias('avgvlu')).show()

+-----+--------+---------+
|stock|totalvlu|   avgvlu|
+-----+--------+---------+
| INFY|440551.0|110137.75|
| BHEL| 94621.0| 23655.25|
+-----+--------+---------+

